In [1]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
#Packages
import requests
import pandas as pd
import zipfile
import os
import io
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection
from arcgis.geometry import SpatialReference, Geometry
import numpy as np
from arcpy.sa import Raster, Reclassify, CostPath, OptimalRegionConnections

In [3]:
origin_destination= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\DoryLocations'

In [4]:
directory=r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb'

In [5]:
dem= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\digital_elevation_model_Clip'
lc= r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\NLCD_2019_Land_Cover_Clip'
bndry = arcpy.ddd.RasterDomain(dem, "aoi", "POLYGON")   

In [7]:
# Clip Landcover to AOI
out_lc =r'C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\landcover_aoi'

lc_aoi = arcpy.management.Clip(lc, "564958.63 4875662.12 577613.63 4889678.12", out_lc, bndry, "255", "ClippingGeometry", "MAINTAIN_EXTENT")
  

In [8]:
# Reclassify Landcover
lc_reclassify = arcpy.sa.Reclassify(lc_aoi, "Value", "1 100 5;101 2;102 2;103 2;104 3;105 2;106 3;107 3;108 4;109 1;110 1", "DATA")
     

In [10]:
# Calculate slope
out_name= r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\slope"
slope = arcpy.sa.Slope(dem, "DEGREE", 1, "PLANAR", "METER")
slope.save(out_name)

RuntimeError: ERROR 010240: Could not save raster dataset to slope with output format FGDBR.

In [11]:
# Reclassify Slope
slope_reclassify = arcpy.sa.Reclassify(slope, "VALUE", "0 2.084853 5;2.084853 3.822231 4;3.822231 6.602035 3;6.602035 18.416204 2;18.416204 88.606262 1", "DATA") 
     

In [12]:
# Path Number 1
weightedslope = Raster("slope_reclassify")*0.34 
weightedlandcover = Raster("lc_reclassify")*0.66


In [13]:
combined_datasets = Raster("weightedslope") + Raster("weightedlandcover")

In [ ]:
arcpy.sa.OptimalRegionConnections(
    in_regions="DoryLocations",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\Optimal_Dory1",
    in_barrier_data=None,
    in_cost_raster="combined_datasets",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

In [16]:
# Path number 2
weightedslope = arcpy.Raster("slope_reclassify") *0.80
weightedlandcover = Raster("lc_reclassify")*0.20
combined_datasets = Raster("weightedslope") + Raster("weightedlandcover")

In [17]:
arcpy.sa.OptimalRegionConnections(
    in_regions="DoryLocations",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\Optimal_DoryLoc2",
    in_barrier_data=None,
    in_cost_raster="combined_datasets",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)


<geoprocessing server result object object at 0x0000025C2B5FE3A0>

In [22]:
# Path number 3
weightedlandcover = Raster("lc_reclassify") *0.50
weightedslope = Raster("slope_reclassify")*0.50
combined_datasets = Raster("weightedslope") + Raster("weightedlandcover")


In [23]:
arcpy.sa.OptimalRegionConnections(
    in_regions="DoryLocations",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\Lab3_Notebooks\Lab3_Notebooks.gdb\Optimal_DoryLoc3",
    in_barrier_data=None,
    in_cost_raster="combined_datasets",
    out_neighbor_paths=None,
    distance_method="PLANAR",
    connections_within_regions="GENERATE_CONNECTIONS"
)

<geoprocessing server result object object at 0x0000025C2B5FEBB0>